# DSS Single Region All DML

- Only calculate single region
- Save all the DML statsitics for two group of samples


In [1]:
import pandas as pd
from pysam import TabixFile
import rpy2
import pathlib
from rpy2.robjects.vectors import IntVector
from collections import defaultdict
%load_ext rpy2.ipython

## Parameters

In [3]:
region = 'chr19:0-5000000'
allc_table_path = 'allc_table.tsv'
smoothing = True

In [4]:
output_path = f'{region}.DSS.DML.hdf'

## R Library

If not installed, run these code to install:
```R
%%R

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DSS")
```

In [5]:
%%R
library(DSS)
require(bsseq)

R[write to console]: Loading required package: Biobase

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort,

## Load Data

In [6]:
allc_table = pd.read_csv(allc_table_path, sep='\t', index_col=0)
allc_table

,sample,group
allc_path,,
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.0.allc.tsv.gz,10_0,A
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.1.allc.tsv.gz,10_1,A
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.2.allc.tsv.gz,10_2,A
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.3.allc.tsv.gz,10_3,B
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.4.allc.tsv.gz,10_4,B
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation/effect_size_0.9/10/10.5.allc.tsv.gz,10_5,B
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation_2/effect_size_0.9/10/10.6.allc.tsv.gz,10_6,C
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation_2/effect_size_0.9/10/10.7.allc.tsv.gz,10_7,C
/gale/netapp/home/hanliu/scratch/DMR_benchmark/simulation_2/effect_size_0.9/10/10.8.allc.tsv.gz,10_8,C


In [7]:
sample_df = allc_table.set_index('sample')
sample_df

,group
sample,
10_0,A
10_1,A
10_2,A
10_3,B
10_4,B
10_5,B
10_6,C
10_7,C
10_8,C


In [16]:
allc_paths = allc_table.index.to_list()
samples = sample_df.index.to_list()

# reformat allc to dss required format
def get_data(allc_paths):
    dss_dfs = []
    for input_path in allc_paths:
        records = []
        with TabixFile(str(input_path)) as f:
            for line in f.fetch(region):
                chromosome, pos, _, _, mc, cov, _ = (line.split('\t'))
                records.append([chromosome, int(pos), int(cov), int(mc)])
        dss_dfs.append(pd.DataFrame(records, columns=['chr', 'pos', 'N', 'X']))
    return dss_dfs

dss_dfs = get_data(allc_paths)

In [17]:
dss_dfs[0]

,chr,pos,N,X
0,chr19,3078948,1,1
1,chr19,3078967,1,1
2,chr19,3079292,1,1
3,chr19,3079293,1,0
4,chr19,3079586,2,2
...,...,...,...,...
40666,chr19,4999371,1,1
40667,chr19,4999647,1,0
40668,chr19,4999648,2,0
40669,chr19,4999926,1,0


In [18]:
group_counts = defaultdict(int)
for sample, dss_df in zip(sample_df.index, dss_dfs):
    group = sample_df.loc[sample, 'group']
    group_counts[group] += dss_df.shape[0]
if any(group_counts.values()):
    # create an empty df in case one of the group is all 0
    dmls = pd.DataFrame([],
                        columns=[
                            'chr', 'pos', 'mu1', 'mu2', 'diff', 'diff.se',
                            'stat', 'phi1', 'phi2', 'pval', 'fdr',
                            'postprob.overThreshold'
                        ])
    # R code will fail in this case, but dmls is created here

In [19]:
group_counts

defaultdict(int, {'A': 122013, 'B': 122013, 'C': 122013, 'D': 122013})

## Create Dataset

In [20]:
%%R -i dss_dfs -i samples
BSobj = makeBSseqData(dss_dfs, samples)
BSobj

An object of type 'BSseq' with
  40671 methylation loci
  12 samples
has not been smoothed
All assays are in-memory


## DML test

In [21]:
%%R
# do not parallel in R
library("BiocParallel")
register(MulticoreParam(1))

In [26]:
%%R -i sample_df -i smoothing
dml_fit <- DMLfit.multiFactor(BSobj, 
                              design=sample_df, 
                              formula=~group,
                              smoothing=smoothing,
                              smoothing.span=500)
dml_fit$X

Fitting DML model for CpG site: 

## Call DML (DMS)

In [34]:
%%R -o dmls
dmls <- DMLtest.multiFactor(dml_fit, term='group')
dmls <- as.data.frame(dmls)

## Save

In [36]:
dmls.to_hdf(output_path, key='data', format="table")

In [37]:
dmls

,chr,pos,stat,pvals,fdrs
1,chr19,3078948,9.287714e-01,0.353008,1.0
2,chr19,3078967,9.287714e-01,0.353008,1.0
3,chr19,3079292,5.687540e-01,0.569523,1.0
4,chr19,3079293,5.687540e-01,0.569523,1.0
5,chr19,3079586,8.620044e-16,1.000000,1.0
...,...,...,...,...,...
40667,chr19,4999371,5.154955e-01,0.606207,1.0
40668,chr19,4999647,4.641193e-02,0.962982,1.0
40669,chr19,4999648,2.774801e-02,0.977863,1.0
40670,chr19,4999926,1.199181e-01,0.904548,1.0
